In [15]:
import numpy as np
import subprocess
import os
import cv2
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract

# From imutils
def resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and grab the image size
    dim = None
    (h, w) = image.shape[:2]
    # if both the width and height are None, then return the original image
    if width is None and height is None:
        return image
    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))
    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)
    # return the resized image
    return resized

#cv2.imshow("quest_tracker", img_rgb)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [16]:
img_rgb = cv2.imread('output_00001.jpg', cv2.IMREAD_UNCHANGED)
img = cv2.cvtColor(np.asarray(img_rgb), cv2.COLOR_BGR2GRAY)

template_rgb = cv2.imread('msq_tracker_2.jpg', cv2.IMREAD_UNCHANGED)
template = cv2.Canny(template_rgb, 50, 200)
(tH, tW) = template.shape[:2]

cv2.imshow("Template", template)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
msq_box = ((tW + 10, 0), (tW + 215, tH))

img_hsv = cv2.cvtColor(np.asarray(img_rgb), cv2.COLOR_BGR2HSV)
template_hsv = cv2.cvtColor(np.asarray(template_rgb), cv2.COLOR_BGR2HSV)
    
lower_red = np.array([100,100,100])
upper_red = np.array([255,255,255])

mask = cv2.inRange(img_hsv, lower_red, upper_red)
template_mask = cv2.inRange(template_hsv, lower_red, upper_red)

res = cv2.bitwise_and(img_hsv, img_hsv, mask= mask)
template_res = cv2.bitwise_and(template_hsv, template_hsv, mask= template_mask)

found = None

for scale in np.linspace(0.2, 1.0, 20)[::-1]:
    resized = resize(res, width = int(res.shape[1] * scale))
    r = res.shape[1] / float(resized.shape[1])
    # if the resized image is smaller than the template, then break
    # from the loop
    if resized.shape[0] < tH or resized.shape[1] < tW:
        break

    result = cv2.matchTemplate(resized, template_res, cv2.TM_CCOEFF)
    (_, maxVal, _, maxLoc) = cv2.minMaxLoc(result)

    if found is None or maxVal > found[0]:
        found = (maxVal, maxLoc, r)
        
(_, maxLoc, r) = found

p1 = np.add((maxLoc[0], maxLoc[1]), msq_box[0])
p2 = np.add((maxLoc[0], maxLoc[1]), msq_box[1])
p1 = (int(p1[0] * r), int(p1[1] * r))
p2 = (int(p2[0] * r), int(p2[1] * r))
cropped = img_rgb[p1[1]:p2[1], p1[0]:p2[0]]
quest_text = pytesseract.image_to_string(cropped).strip().lstrip().rstrip()
print(quest_text)

cv2.rectangle(img_rgb, p1, p2, (0, 0, 255), 2)
cv2.imshow("Image", img_rgb)
cv2.waitKey(0)

Coming to Ishgard


-1